In [21]:
! pip install streamlit -q

In [20]:
pip install pandas_ta


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=818611924c47c77b198f068cb2918d6b9287170ecbcae7866f03fe292789ae61
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [22]:
!pip install pyngrok


In [39]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
from alpha_vantage.fundamentaldata import FundamentalData
from stocknews import StockNews
import pandas_ta as ta
from pandas.tseries.holiday import USFederalHolidayCalendar

# Function to set a background image
def set_background(background_image):
    st.markdown(
        f"""
        <style>
        .stApp {{
            background: url("{background_image}");
            background-size: cover
        }}
        </style>
        """,
        unsafe_allow_html=True
    )

# Set background image
set_background("https://images.unsplash.com/photo-1578861082890-f9211a299fb6")

# Add a famous quote
st.markdown(
    """
    <div style='text-align: center; font-size: 24px; font-weight: bold;'>
        YOUR TRADING IS GOING TO PAY OFF SOONER THAN YOU THINK. BE PATIENT.
    </div>
    """,
    unsafe_allow_html=True
)

st.title('Stock Dashboard')

# List of stocks
stocks = ['AAPL', 'GOOG', 'MSFT', 'AMZN', 'META', 'NFLX', 'TSLA', 'BA', 'IBM', 'NVDA',
          'INTC', 'CSCO', 'PYPL', 'ADBE', 'NFLX', 'ASML', 'CMCSA', 'PEP', 'AVGO', 'TXN']

# Function to analyze stocks and recommend the one with the least error
def recommend_stock(stocks):
    min_error = float('inf')
    recommended_stock = None
    for stock in stocks:
        try:
            data = yf.download(stock, period="1y")
            if data.empty:
                continue
            error = np.std(data['Close']) / np.mean(data['Close'])
            if error < min_error:
                min_error = error
                recommended_stock = stock
        except Exception as e:
            st.write(f"Error downloading data for {stock}: {e}")
    return recommended_stock

# Recommended stock to buy
recommended_stock = recommend_stock(stocks)
st.sidebar.write(f"Recommended Stock to Buy: {recommended_stock}")

# Input sidebar for ticker, start and end date
selected_stock = st.sidebar.selectbox('Select Stock', stocks, index=0)
start_date = st.sidebar.text_input('Start Date', '2001-01-01')
end_date = st.sidebar.text_input('End Date', '2024-03-29')

# Download data function call
data = yf.download(selected_stock, start=start_date, end=end_date)

# Plot the stock price line chart
fig = px.line(data, x=data.index, y=data['Adj Close'], title=selected_stock)
st.plotly_chart(fig)

# Display Price Movements
st.header('Price Movements')
data2 = data.copy()
data2['% Change'] = data['Adj Close'].pct_change()
data2.dropna(inplace=True)
st.write(data2)

annual_return = data2['% Change'].mean() * 252 * 100
st.write('Annual Return is', annual_return, '%')
stdev = np.std(data2['% Change']) * np.sqrt(252)
st.write('Standard Deviation is ', stdev * 100, '%')

# Fundamental Data
st.header('Fundamental Data')
key = 'N2DNGUJU8ZBKH0VJ'  # Add your Alpha Vantage API key here
fd = FundamentalData(key, output_format='pandas')
st.subheader('Balance Sheet')
balance_sheet = fd.get_balance_sheet_annual(selected_stock)[0]
bs = balance_sheet.T[2:]
bs.columns = list(balance_sheet.T.iloc[0])
st.write(bs)

st.subheader('Income Statement')
income_statement = fd.get_income_statement_annual(selected_stock)[0]
isl = income_statement.T[2:]
isl.columns = list(income_statement.T.iloc[0])
st.write(isl)

st.subheader('Cash Flow Statement')
cash_flow = fd.get_cash_flow_annual(selected_stock)[0]
cf = cash_flow.T[2:]
cf.columns = list(cash_flow.T.iloc[0])
st.write(cf)

# Top 10 News for Recommended Stock
st.header('Top 10 News for Recommended Stock')
sn = StockNews(recommended_stock)
df_news = sn.read_rss()
for i in range(10):
    st.subheader(f'News {i+1}')
    st.write(df_news['published'][i])
    st.write(df_news['title'][i])
    st.write(df_news['summary'][i])
    title_sentiment = df_news['sentiment_title'][i]
    st.write(f'Title Sentiment: {title_sentiment}')
    news_sentiment = df_news['sentiment_summary'][i]
    st.write(f'News Sentiment: {news_sentiment}')

# Technical Analysis Dashboard
st.header('Technical Analysis Dashboard')
indicators = ['MACD', 'RSI', 'Bollinger Bands', 'ATR', 'OBV', 'AD', 'CCI', 'Chaikin Money Flow', 'VWAP']
selected_indicator = st.selectbox('Select Technical Indicator', options=indicators)

indicator_data = None
if selected_indicator == 'MACD':
    indicator_data = ta.macd(data['Close'])
elif selected_indicator == 'RSI':
    indicator_data = ta.rsi(data['Close'])
elif selected_indicator == 'Bollinger Bands':
    indicator_data = ta.bbands(data['Close'])
elif selected_indicator == 'ATR':
    indicator_data = ta.atr(data['High'], data['Low'], data['Close'])
elif selected_indicator == 'OBV':
    indicator_data = ta.obv(data['Close'], data['Volume'])
elif selected_indicator == 'AD':
    indicator_data = ta.ad(data['High'], data['Low'], data['Close'], data['Volume'])
elif selected_indicator == 'CCI':
    indicator_data = ta.cci(data['High'], data['Low'], data['Close'])
elif selected_indicator == 'Chaikin Money Flow':
    indicator_data = ta.cmf(data['High'], data['Low'], data['Close'], data['Volume'])
elif selected_indicator == 'VWAP':
    indicator_data = ta.vwap(data['High'], data['Low'], data['Close'], data['Volume'])

if indicator_data is not None:
    st.line_chart(indicator_data)
    st.write(indicator_data)

# Portfolio Summary
st.header('Portfolio Summary')
if st.checkbox("Show Portfolio Summary"):
    # Example data (replace with your actual portfolio data)
    portfolio_value = 1000000  # Total portfolio value
    cash = 200000  # Cash balance
    stocks_value = 800000  # Value of stocks
    total_return = 0.15  # Total return (15%)
    annual_return = 0.10  # Annualized return (10%)
    volatility = 0.20  # Portfolio volatility (20%)

    st.subheader("Portfolio Value")
    st.write(f"Total Portfolio Value: ${portfolio_value:,.2f}")
    st.write(f"Cash: ${cash:,.2f}")
    st.write(f"Stocks Value: ${stocks_value:,.2f}")

    st.subheader("Performance Metrics")
    st.write(f"Total Return: {total_return * 100:.2f}%")
    st.write(f"Annualized Return: {annual_return * 100:.2f}%")
    st.write(f"Portfolio Volatility: {volatility * 100:.2f}%")

# Watchlist
st.header('Watchlist')
if st.checkbox("Show Watchlist"):
    # Example data (replace with your actual watchlist data)
    watchlist = {
        'AAPL': {'price': 150.50, 'change': 1.25, 'percent_change': 0.84},
        'GOOG': {'price': 2800.75, 'change': -5.50, 'percent_change': -0.20},
        'MSFT': {'price': 250.20, 'change': 2.80, 'percent_change': 1.13},
        # Add more stocks as needed
    }

    st.subheader("Stocks in Watchlist")
    for stock, data in watchlist.items():
        st.write(f"{stock}:")
        st.write(f"Price: ${data['price']:.2f}")
        st.write(f"Change: ${data['change']:.2f}")
        st.write(f"Percent Change: {data['percent_change']:.2f}%")

# Market Indices
st.header('Market Indices')
if st.checkbox("Show Market Indices"):
    # Example data (replace with your actual market indices data)
    sp500_value = 4000  # S&P 500 index value
    dow_value = 33000  # Dow Jones Industrial Average index value
    nasdaq_value = 13000  # NASDAQ index value

    st.subheader("Market Indices")
    st.write(f"S&P 500 Index: {sp500_value}")
    st.write(f"Dow Jones Industrial Average: {dow_value}")
    st.write(f"NASDAQ Composite: {nasdaq_value}")

# Individual Stock Performance
st.header('Individual Stock Performance')
if st.checkbox("Show Individual Stock Performance"):
    for symbol in stocks:
        # Replace this with your actual stock data retrieval and calculations
        # For now, show a placeholder
        st.subheader(f"{symbol} Performance")
        st.write("Placeholder for performance metrics")

        st.subheader(f"{symbol} Performance")
        st.write("Placeholder for performance metrics")

# Stock Market Chatbot
st.header(' Chatbot')
st.subheader('Ask queries related to Stock Market')

user_input = st.text_input('Type Your Question: ', '')

if user_input.lower() in ['hi', 'hello', 'hey']:
    st.write('Bot: Hello! How can I help you today?')

elif user_input.lower() in ['what is stock market', 'define stock market']:
    st.write('Bot: The stock market refers to the collection of markets and exchanges where activities like buying, selling, and issuance of shares of publicly-held companies take place.')

elif user_input.lower() in ['what is stock market prediction', 'define stock market prediction']:
    st.write('Bot: Stock market prediction involves forecasting future movements of stock prices based on historical data and market trends.')

elif user_input.lower() in ['what is a stock', 'define stock']:
    st.write('Bot: A stock represents a share in the ownership of a company. Owning stock means owning a piece of the company.')

elif user_input.lower() in ['thank you', 'thanks', 'bye', 'goodbye']:
    st.write('Bot: You\'re welcome! Have a great day!')

else:
    st.write('Bot: Sorry, I didn\'t get that. Please ask something else.')

# Footer
st.markdown(
    """
    <style>
    .footer {
        position: fixed;
        left: 0;
        bottom: 0;
        width: 100%;
        background-color: #f0f0f0;
        color: black;
        text-align: center;
        font-size: 12px;
    }
    </style>
    """
    , unsafe_allow_html=True
)

st.markdown(
    """
    <div class="footer">
    <p>Created by JOEL</p>
    </div>
    """
    , unsafe_allow_html=True
)


Overwriting app.py


In [25]:
!pip install pandas_market_calendars



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.6/192.6 kB 14.7 MB/s eta 0:00:00


In [26]:
!pip install alpha_vantage


In [27]:
!pip install yfinance


In [28]:

!pip install stocknews

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stocknews: filename=stocknews-0.9.11-py3-none-any.whl size=5396 sha256=25f6eb90b2f91cf4653a4de06f6aff80134a9f8831d4c5f567d4a6e1fa54e5d5
  Stored in directory: /root/.cache/pip/wheels/9e/59/42/c5ab3171c3509ff8bc6aafbc089fe826eacf531ef7f7803ff6
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6049 sha256=b992c90c5033c593d01190fe56c561c9cc5aa83acb1bb55d25e61b2d0384343b
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built stocknews sgmllib3k


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.245.24.22:8501

npx: installed 22 in 2.184s
your url is: https://some-houses-tap.loca.lt
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 com

In [29]:
!pip install alpha_vantage


In [30]:
!pip install stocknews
